### First Regression (Turnover intention ~ unfair treatment x neg. reciprocity)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

### 1.  Read in SOEP Data:
- vp : 2005 data : main variables of interest: questions on negative reciprocity
- wp: 2006 data : main variables of interest: question on perceived recognition for work
- xp: 2007 data : main variables of interest: turnover intentions, controls

In [ ]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
## copy your path in here:
# Path Max: C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw
# Path Maxie: /Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw

data_folder = Path("C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp', 'xp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [ ]:
# read in 2005 data for the reciprocity measures
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_2005 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})
# create dummies for take_revenge question
# Create dummy variables
dummies = pd.get_dummies(df_2005['similar_problems'])

# Join the dummy variables to the original dataframe
df_2005 = pd.concat([df_2005, dummies], axis=1)
# rename dummy
df_05 = df_2005.rename(columns = {'[1] Trifft ueberhaupt nicht zu' : 'rec1' , '[2] Skala 1-7' : 'rec2' , '[3] Skala 1-7' : 'rec3' ,'[4] Skala 1-7' : 'rec4' ,'[5] Skala 1-7' : 'rec5' ,'[6] Skala 1-7' : 'rec6' ,'[7] Trifft voll zu' : 'rec7'})
df_05 = df_05.drop(columns = ["[-1] keine Angabe"])

In [ ]:
# read in 2006 data
# personal = personal advancement
# still includes all unfair treat
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear", 'wp43b01', 'wp43b02', 'wp43b03', 'wp43b04', 'wp43b05', 'wp43b06', 'wp43b07','wp43b08' ]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={ 'wp43b01': 'recog_sup', 'wp43b02': 'felt_recog_sup',"wp43b03": "recog_effort",  'wp43b04': 'felt_recog_effort', "wp43b05": "recog_personal", "wp43b06" :"felt_recog_personal" ,"wp43b07": "recog_pay",'wp43b08': 'felt_recog_pay'})

In [ ]:
#read in 2007 data
# here left out 'xp8601' for school degree since we have it in another module also 'xp0102' : 'work_satisfaction' for the beginning
#for outcome and all controls
data3= pd.read_stata(file_paths[2], columns=["pid", "hid", "syear", 'xp13101' , 'xp13102', 'xp2701', 'xp7302','xp7202','xp28', 'xp3001' ,  'xp5701' , 'xp0102']).set_index(['pid', 'hid'])
df_07 = data3.rename(columns= {'xp13101':'gender','xp13102': 'year_birth' ,'xp2701': 'turnover_intention' , 'xp7302': 'wage_lastmonth','xp7202': 'overtime','xp28': 'new_job', 'xp3001': 'reason_new_job',  'xp5701' : 'commute_distance' , "xp0102" : "work_satisfaction"})

In [ ]:
# read in 2007 data from work module

# adapt path and merge
hours07 = pd.read_stata(file_paths_2[2], columns=["pid","hid", "syear", "xvebzeit", "xpsbil","nace07" ,"betr07", "xerwzeit", "xbilzeit"]).set_index(['pid', 'hid'])
work07 = hours07.rename(columns={'xvebzeit': 'working_hours', "xpsbil": "school_degree", "nace07": "sector","betr07": "firmsize", "xerwzeit": "tenure" , "xbilzeit" : "years_educ"})

 ### 2. Define Functions and mappings for cleaning data

In [ ]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}
## mapping for recognition questions: binary -> binary -> unfair treatment: No -> later 1 fair treatment: Yes -> 0
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 2,
    '[2] Nein': 1,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}
# mapping new job into binary variable
new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1, 
    '[1] Ja': 1, 
    '[2] Nein': 2,
    '[3] Ja, nach Datenpruefung': 1,
}
# mapping for job satisfaction: split up into binary with roughly equal value counts for simplicity: might change that later to categories,
satisfaction_mapping = {
    '[0] 0 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[1] 1 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[2] 2 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[3] 3 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[4] 4 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[5] 5 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[6] 6 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[7] 7 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[8] 8 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[9] 9 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[10] 10 Zufrieden: Skala 0-Niedrig bis 10-Hoc': 1,
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
}
turnover_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 2,
    '[10] 10% wahrscheinlich': 1,
    '[20] 20% wahrscheinlich': 1,
    '[30] 30% wahrscheinlich': 1,
    '[40] 40% wahrscheinlich': 1, 
    '[50] 50% wahrscheinlich': 1,
    '[60] 60% wahrscheinlich': 1,
    '[70] 70% wahrscheinlich': 1,
    '[80] 80% wahrscheinlich': 1,
    '[90] 90% wahrscheinlich': 1,
    '[100] 100% wahrscheinlich': 1,
}
# mapping for turnover intention robustness check: Cardinal
turnover_mapping_cardinal = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 0,
    '[10] 10% wahrscheinlich': 10,
    '[20] 20% wahrscheinlich': 20,
    '[30] 30% wahrscheinlich': 30,
    '[40] 40% wahrscheinlich': 40, 
    '[50] 50% wahrscheinlich': 50,
    '[60] 60% wahrscheinlich': 60,
    '[70] 70% wahrscheinlich': 70,
    '[80] 80% wahrscheinlich': 80,
    '[90] 90% wahrscheinlich': 90,
    '[100] 100% wahrscheinlich': 100,
}
# mapping for new job to easier remove negatives
reason_new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1, 
    '[1] Erstmals erwerbstaetig': 1,
    '[2] Wieder erwerbstaetig': 2,
    '[3] Stelle bei neuen Arbeitgeber': 3,  
    '[4] Uerbnommen von Betrieb': 4,
    '[5] Stellenwechsel im Betrieb': 5, 
    '[6] Selbstaendig geworden': 6,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# sector_map
sector_map = {
    "[1] Landwirtschaft und  Jagd": 1,
    "[2] Forstwirtschaft": 2,
    "[5] Fischerei und Fischzucht": 5,
    "[10] Kohlenbergbau, Torfgewinnung": 10,
    "[11] Gewinnung von Erdöl und Erdgas, Erbringung damit verbundener Dienstleistungen": 11,
    "[12] Bergbau auf Uran- und Thoriumerze": 12,
    "[13] Erzbergbau": 13,
    "[14] Gewinnung von Steinen und Erden, sonstiger Bergbau": 14,
    "[15] Herstellung von Nahrungs- und Futtermitteln sowie Getränken": 15,
    "[16] Tabakverarbeitung": 16,
    "[17] Herstellung von Textilien": 17,
    "[18] Herstellung von Bekleidung": 18,
    "[19] Herstellung von Leder und Lederwaren": 19,
    "[20] Herstellung von Holz sowie Holz-, Kork- und Flechtwaren (ohne Herstellung von Möbeln)": 20,
    "[21] Herstellung von Papier, Pappe und Waren daraus": 21,
    '[22] Herstellung von Verlags- und Druckerzeugnissen,  Vervielfältigung von bespielten Ton-, Bild- und Datenträgern': 22,
    "[23] Kokerei, Mineralölverarbeitung, Herstellung und Verarbeitung von Spalt- und Brutstoffen": 23,
    "[24] Herstellung von chemischen Erzeugnissen": 24,
    "[25] Herstellung von Gummi- und Kunststoffwaren": 25,
    "[26] Herstellung von Glas und Glaswaren, Keramik, Verarbeitung von Steinen und Erden": 26,
    "[27] Metallerzeugung und -bearbeitung": 27,
    "[28] Herstellung von Metallerzeugnissen": 28,
    "[29] Maschinenbau": 29,
    "[31] Herstellung von Geräten der Elektrizitätserzeugung, -verteilung u. Ä.": 31,
    "[30] Herstellung von Büromaschinen, Datenverarbeitungsgeräten und -einrichtungen": 30,
    "[32] Rundfunk- und Nachrichtentechnik": 32,
    "[33] Medizin-, Mess-, Steuer- und Regelungstechnik, Optik, Herstellung von Uhren": 33,
    "[34] Herstellung von Kraftwagen und Kraftwagenteilen": 34,
    "[35] Sonstiger Fahrzeugbau": 35,
    "[36] Herstellung von Möbeln, Schmuck, Musikinstrumenten, Sportgeräten, Spielwaren und sonstigen Erzeugnissen": 36,
    "[37] Rückgewinnung": 37,
    "[40] Energieversorgung": 40,
    "[41] Wasserversorgung": 41,
    "[45] Bau": 45,
    "[50] Kraftfahrzeughandel; Instandhaltung und Reparatur von Kraftfahrzeugen; Tankstellen": 50,
    "[51] Handelsvermittlung und Großhandel (ohne Handel mit Kraftfahrzeugen)": 51,
    "[52] Einzelhandel (ohne Handel mit Kraftfahrzeugen und ohne Tankstellen); Reparatur von Gebrauchsgütern": 52,
    "[55] Beherbergungs- und Gaststätten": 55,
    "[60] Landverkehr; Transport in Rohrfernleitungen": 60,
    "[61] Schifffahrt": 61,
    "[62] Luftfahrt": 62,
    "[63] Hilfs- und Nebentätigkeiten für den Verkehr; Verkehrsvermittlung": 63,
    "[64] Nachrichtenübermittlung": 64,
    "[65] Kreditinstitute": 65,
    "[66] Versicherungen (ohne Sozialversicherung)": 66,
    "[67] Mit den Kreditinstituten und Versicherungen verbundene Tätigkeiten": 67,
    "[70] Grundstücks- und Wohnungswesen": 70,
    "[71] Vermietung beweglicher Sachen ohne Bedienungspersonal": 71,
    "[72] Datenverarbeitung und Datenbanken": 72,
    "[73] Forschung und Entwicklung": 73,
    "[74] Erbringung von unternehmensbezogenen Dienstleistungen": 74,
    "[75] Öffentliche Verwaltung, Verteidigung, Sozialversicherung": 75,
    "[80] Erziehung und Unterricht": 80,
    "[85] Gesundheits-, Veterinär- und Sozialwesen": 85,
    "[90] Abwasser- und Abfallbeseitigung und sonstige Entsorgung": 90,
    "[91] Interessenvertretungen sowie kirchliche und sonstige Vereinigungen (ohne Sozialwesen, Kultur und Sport)": 91,
    "[92] Kultur, Sport und Unterhaltung": 92,
    "[93] Erbringung von sonstigen Dienstleistungen": 93,
    "[95] Private Haushalte mit Hauspersonal": 95,					
    "[96] Industrie - ohne weitere Zuordnung": 96,					
    "[97] Handwerk - ohne weitere Zuordnung": 97,					
    "[98] Dienstleistungen ohne weitere Zuordnung": 98,					
    "[99] Exterritoriale Organisationen und Körperschaften": 99,				
    "[100] Produzierendes Gewerbe ohne w.Zuordnung": 100,
    "[-1] keine Angabe": 3,
    '[-2] trifft nicht zu': 0, 
    "[-3] unplausibler Wert": -3,
    "[-4] unzulaessige Mehrfachantwort": -4, 
    "[-5] in Fragebogenversion nicht enthalten": -5,
    "[-6] Fragebogenversion mit geaenderter Filterfuehrung": -6, 
    "[-7] nur in weniger eingeschraenkter Edition verfuegbar": -7,
    "[-8] Frage in diesem Jahr nicht Teil des Frageprogramms": -8,
}
# reversed mapping to redo changes
reversed_mapping_reason = {v: k for k, v in reason_new_job_mapping.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}
reversed_mapping_sector = {v: k for k, v in sector_map.items()}
# mapping for binary reciprocity measure
def binary_reciprocity(x):
    if x >= 3:
        return 1 # High Group
    else:
        return 0 # Low Group

## function for recoding values and dropping missing

def recode_categoricals(inputdf,rc_cardinal = 0):
    """
        merges data from different years, applies recoding to categoricals and constructs additional variables.

        Input:
            - inputdf : merged dataframe
            - rc_cardinal: optional argument: if == 1 turnover intentions will be coded as cardinal instead of binary 
            - rc_rec_binary: optional argument: if == 1 neg. reciprocity will be split into 2 groups: high and low
        Output:
            - df : cleaned Dataframe

    """
    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode new job reason variable
    merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
    merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode sector
    merged['sector'] = merged['sector'].map(sector_map)
    # recode job satisfaction
    merged['work_satisfaction']= merged['work_satisfaction'].map(satisfaction_mapping)
    # recode turnover intention variable
    if rc_cardinal == 1:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping_cardinal)
    else:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping)
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    
    output = merged
    return output

# Merge dataframes: a bit tough to read as its nested, merges 4 dataframes: 2005,2006,2007 and 2007gen

def merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal = 0,rc_rec_binary=0):
    """
    merges data from different years, applies recoding to categoricals and constructs additional variables.

    Input:
        - df_05 : Pd.Dataframe contains reciprocity measures
        - df_06 : Pd.Dataframe contains unfair treatment measures
        - df_07 : Pd.Dataframe contains outcome and controls
        - work07 : Pd.Dataframe contains additional controls

    Output:
        - df : cleaned Dataframe

    """
    allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
    recoded = recode_categoricals(allmerged_df,rc_cardinal).astype('int')
    # replaces negative values with n.a.n 
    recoded = recoded.mask(recoded < 0, np.nan)
    # recode sector back intro category
    
    recoded["wage_lastmonth"] = np.log(recoded["wage_lastmonth"])
    recoded = recoded[recoded["wage_lastmonth"] != -np.inf] 
    # construct avg reciprocity measure
    recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
    # For robustness check: option to construct binary reciprocity measure.
    if rc_rec_binary == 1:
        recoded['binary_rec'] = recoded['avg_rec'].apply(binary_reciprocity)
    
    # construct age, potential experience and age^2
    recoded['age'] = 2007 - recoded['year_birth']
    recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
    recoded["age_squared"] = (recoded["age"] ** 2) / 100
    recoded["tenure_squared"] = (recoded["tenure"] ** 2) / 100
    # recode categoricals back to make it better readable
    recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
    recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)
    recoded["sector"] = recoded["sector"].map(reversed_mapping_sector)
    # transform binary variables with 1 and 2 into 1 and 0
    columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "turnover_intention", "new_job"]

    # Iterate over the columns and replace the values 2 with 0 
    for col in columns_to_transform:
        recoded[col] = recoded[col].replace({2: 0})

    # save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
    df = recoded
    
    return df

# Add Mincer Wage Regression and adds its residuals to the Dataframe

def add_mincer_residuals(cleaneddata):
    
    df_cleaned = cleaneddata
    # specify which columns to drop from our dataframe
    df_mincer = cleaneddata.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'syear_y', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup', 'age', 'commute_distance', 'recog_effort', 'working_hours', 'turnover_intention', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction' , 'sector' , 'tenure_squared'])
    # Convert 'gender' and 'sector' columns to categorical data type
    for col in ['gender']:
        df_mincer[col] = df_mincer[col].astype('category')
    df_mincer = df_mincer.dropna()
    # Define the dependent variable
    y = df_mincer['wage_lastmonth']
    # Define the independent variables
    X = df_mincer[['gender', 'firmsize', 'tenure', 'years_educ', 'potential_experience', 'age_squared']]

    # Add a constant term to the independent variables
    X = sm.add_constant(X)

    # Fit the Mincer wage regression model
    mincer_model = sm.OLS(y, X).fit()
    
    # Create a new column in the dataframe with the same name as the residuals array/ delete relative wage entries
    df_cleaned['mincer_residuals'] = None
    # Match the rows of the dataframe with the values in the residuals array using the index
    df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid

    return df_cleaned

### Inlcude People who switched their jobs in 2006-2007 with 2006 controls

In [ ]:
def include_jobchangers(data07,onlynewemployer=True,worksatisfaction=False,rc_cardinal = False):
    """
    Input: 
            df: finished 2007 dataframe
            onlynewemployer: optional argument: If True only includes those who switched jobs to a new employer
            worksatisfaction: if True drops observations where job satisfaction between 2006 and 2007 changed to check for our Assumption
    
    """
    df = data07
    # read in cleaned 2006 dataset:
    #  change path here : Maxie: '/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/2006jobchange.csv'
    #                     Max 'C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv'  
    observations_2006 = pd.read_csv('C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv')
    # some initial datamanagement
    observations_2006.reset_index(inplace=True)
    observations_2006.set_index(["pid","hid"], inplace=True)
    observations_2006.drop(columns=observations_2006.filter(regex='^syear').columns, inplace=True)
    observations_2006.drop("index",axis=1 ,inplace =True)
    # If cardinal turnover intentions are coded to 100
    if rc_cardinal == True:
        observations_2006["turnover_intention"] = 100
    # drop s year columns from both dataframes
    df.drop(columns=df.filter(regex='^syear').columns, inplace=True)
    
    # optional worksatisfaction robustness check
    if worksatisfaction == True:
        satisfaction = pd.DataFrame(df['work_satisfaction']).join(pd.DataFrame(observations_2006['work_satisfaction']), on = ["pid", "hid"], lsuffix = "_07" , rsuffix ="_06")
        # list of IDs where job satisfaction changed over the past year
        ID_keep_satis = satisfaction[satisfaction["work_satisfaction_07"] == satisfaction["work_satisfaction_06"]].index
        # drops all persons from 2007 dataframe where job satisfaction changed
        df.drop(df[~df.index.isin(ID_keep_satis)].index, inplace=True)
    
    # create list of IDs of people who switched to a new employer in the last year in 2007
    if onlynewemployer == True:
        IDs_tokeep = df[(df["new_job"] == 1) & (df["reason_new_job"] == '[3] Stelle bei neuen Arbeitgeber')].index
    # if True drops people whose work satisfaction changed between 2006 and 2007
    else:
        IDs_tokeep = df[(df["new_job"] == 1)].index
    
    # drop all who changed their job in 2007 dataframe and replace 2007 controls with 2006 controls
    df.drop(df[df["new_job"] == 1].index, inplace = True)
    observations_2006.drop(observations_2006[~observations_2006.index.isin(IDs_tokeep)].index, inplace=True)
    #concat both dataframes
    dfconcat = pd.concat([df,observations_2006])
    
    return dfconcat

### 3. Analysis
#### OLS

- Probit Estimation: TurnoverIntention_{2005} = Constant + Neg-Rec + Unfair + Rec X Unfair + Controls + Error

- As the measure for unfair treatment in the first regression i first used recog_effort -> decide for one later
    - "When I consider all my accomplishments and efforts, the recognition of I've received seems about right to me"

In [ ]:
df_OLS = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# add interaction term
df_OLS["recXrecog_effort"] = df_OLS["recog_effort"] * df_OLS["avg_rec"]
## Drop n.a.n.s
df_OLS.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'], inplace=True)

df_OLS.dropna(inplace=True)
# drop missing data in regression dataframe


import statsmodels.formula.api as smf



# Specify model
formula_main = 'turnover_intention ~ recog_effort + avg_rec + recXrecog_effort + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals'

# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=df_OLS).fit(cov_type='cluster', cov_kwds={'groups': df_OLS['sector']})

# Print the regression results
reg.summary()

### Robustness Checks


#### 1. Recode turnover variable into cardinal variable: Optional argument in recode_categoricals

In [387]:
# optional argument ==1
df_card = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal=1)),rc_cardinal=True)
## drop n.a.ns
df_card.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth','rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' ,'work_satisfaction'], inplace = True)
# add interaction term
df_card["recXrecog_effort"] = df_card["recog_effort"] * df_card["avg_rec"]
# drop missing data in regression dataframe
df_card.dropna(inplace=True)

import statsmodels.formula.api as smf

# Specify model
formula_main = 'turnover_intention ~ recog_effort + avg_rec + recXrecog_effort + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals'

# Fit the regression and cluster on the sector variable
reg_cardinal = smf.ols(formula_main, data=df_card).fit(cov_type='cluster', cov_kwds={'groups': df_card['sector']})

# Print the regression results
reg_cardinal.summary()

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-354-02c81fd1d7bc>:317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.258
Model:                            OLS   Adj. R-squared:                  0.257
Method:                 Least Squares   F-statistic:                     365.1
Date:                Thu, 12 Jan 2023   Prob (F-statistic):           7.74e-50
Time:                        15:21:28   Log-Likelihood:                -27270.
No. Observations:                5713   AIC:                         5.457e+04
Df Residuals:                    5700   BIC:                         5.465e+04
Df Model:                          12                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                5.3321      0.635      8.403      0.000       4.088       6.576
recog_effort            10.1030      2.782      3.631      0.000       4.650      15.556
avg_rec                  0.6029      0.344      1.755      0.079      -0.070       1.276
recXrecog_effort        -0.5891      0.744     -0.792      0.428      -2.046       0.868
working_hours            0.1093      0.058      1.876      0.061      -0.005       0.223
firmsize                -0.4938      0.190     -2.599      0.009      -0.866      -0.121
tenure                  -2.8697      0.181    -15.836      0.000      -3.225      -2.514
tenure_squared           5.7427      0.438     13.123      0.000       4.885       6.600
age                      4.9993      0.504      9.917      0.000       4.011       5.987
age_squared            -15.4747      1.876     -8.247      0.000     -19.152     -11.797
years_educ               0.9819      0.226      4.340      0.000       0.538       1.425
commute_distance         0.0357      0.008      4.260      0.000       0.019       0.052
potential_experience     0.1517      0.022      6.876      0.000       0.108       0.195
mincer_residuals        -7.4212      1.092     -6.794      0.000      -9.562      -5.280
==============================================================================
Omnibus:                      554.228   Durbin-Watson:                   1.528
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              724.953
Skew:                           0.848   Prob(JB):                    3.79e-158
Kurtosis:                       3.414   Cond. No.                     7.29e+17
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 9.47e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### 2. Creating dummies and include 7 interaction terms : Decide for one of the 3 questions for simplicity

When treating the categories as numerical: we are making assumptions about the differences between the scale items. If those distances can be considered equal at all levels, then it is reasonable to treat reciprocity as numerical. (i.e a one unit change from 1 to 2 is equivalent to a one unit change from 6 to 7)


For dummy coding we need to exclude one of the categories in the dataframe and make it the reference category: This will be the lowest level of reciprocity 1 and will be coded as zero.  so rec2 rec3 rec4 , ... and their interaction terms with unfair treatment stay in the regression.

rec2 is then interpreted as the mean of turnover intentions in the rec2 group - the mean of turnover intentions in the rec1 group (reference group) holding everything else constant

In [388]:
# 1. optional argument:  rc_binary ==1
dfrc_dummy = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# specify columns which we need for regression by dropping everything else
dfrc_dummy.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay','year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'avg_rec','rec1', 'work_satisfaction'], inplace = True)
# add interaction terms
for col in ['rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7']:
    dfrc_dummy = dfrc_dummy.assign(**{col + '_X_recog_effort': dfrc_dummy[col] * dfrc_dummy['recog_effort']})
# drop missing data in regression dataframe
dfrc_dummy = dfrc_dummy.dropna()


# Specify model
formula_main = 'turnover_intention ~ recog_effort + rec2 + rec3 + rec4 + rec5 + rec6 + rec2_X_recog_effort + rec3_X_recog_effort + rec4_X_recog_effort + rec5_X_recog_effort + rec6_X_recog_effort + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals'

# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=dfrc_dummy).fit(cov_type='cluster', cov_kwds={'groups': dfrc_dummy['sector']})

# Print the regression results
reg.summary()

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-354-02c81fd1d7bc>:317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     656.5
Date:                Thu, 12 Jan 2023   Prob (F-statistic):           2.83e-60
Time:                        15:21:33   Log-Likelihood:                -3454.4
No. Observations:                5713   AIC:                             6951.
Df Residuals:                    5692   BIC:                             7090.
Df Model:                          20                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0354      0.006      5.500      0.000       0.023       0.048
recog_effort             0.1468      0.024      6.084      0.000       0.099       0.194
rec2                     0.1414      0.015      9.343      0.000       0.112       0.171
rec3                     0.1438      0.022      6.504      0.000       0.100       0.187
rec4                     0.1222      0.021      5.760      0.000       0.081       0.164
rec5                     0.1551      0.028      5.505      0.000       0.100       0.210
rec6                     0.1706      0.040      4.307      0.000       0.093       0.248
rec2_X_recog_effort     -0.0410      0.031     -1.340      0.180      -0.101       0.019
rec3_X_recog_effort     -0.0079      0.040     -0.196      0.845      -0.087       0.071
rec4_X_recog_effort     -0.0811      0.037     -2.220      0.026      -0.153      -0.010
rec5_X_recog_effort     -0.0758      0.043     -1.745      0.081      -0.161       0.009
rec6_X_recog_effort     -0.0713      0.073     -0.980      0.327      -0.214       0.071
working_hours            0.0020      0.001      1.789      0.074      -0.000       0.004
firmsize                -0.0118      0.004     -3.045      0.002      -0.019      -0.004
tenure                  -0.0219      0.002    -11.269      0.000      -0.026      -0.018
tenure_squared           0.0326      0.004      8.211      0.000       0.025       0.040
age                      0.0422      0.005      8.164      0.000       0.032       0.052
age_squared             -0.0995      0.019     -5.209      0.000      -0.137      -0.062
years_educ               0.0167      0.007      2.547      0.011       0.004       0.030
commute_distance         0.0004   9.52e-05      4.291      0.000       0.000       0.001
potential_experience     0.0006      0.000      2.784      0.005       0.000       0.001
mincer_residuals        -0.0803      0.019     -4.310      0.000      -0.117      -0.044
==============================================================================
Omnibus:                     5207.270   Durbin-Watson:                   1.808
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              356.887
Skew:                          -0.104   Prob(JB):                     3.18e-78
Kurtosis:                       1.793   Cond. No.                     2.37e+17
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 8.98e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

##### Robustness Check : Drop people whose job satisfaction level changed

In [389]:
dfworksatisfation = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)),worksatisfaction = True)
# specify columns which we need for regression by dropping everything else

df_rc_ws = dfworksatisfation.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' ,'work_satisfaction'])
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month """
# add interaction term
df_rc_ws["recXrecog_effort"] = df_rc_ws["recog_effort"] * df_rc_ws["avg_rec"]
# drop missing data in regression dataframe
df_rc_ws.dropna(inplace=True)

formula_main = 'turnover_intention ~ recog_effort + avg_rec + recXrecog_effort + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals'
# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=df_rc_ws).fit(cov_type='cluster', cov_kwds={'groups': df_rc_ws['sector']})

# Print the regression results
reg.summary()

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-354-02c81fd1d7bc>:317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.187
Method:                 Least Squares   F-statistic:                     412.0
Date:                Thu, 12 Jan 2023   Prob (F-statistic):           1.41e-50
Time:                        15:21:36   Log-Likelihood:                -2829.3
No. Observations:                4557   AIC:                             5685.
Df Residuals:                    4544   BIC:                             5768.
Df Model:                          12                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0368      0.007      4.931      0.000       0.022       0.051
recog_effort             0.1951      0.031      6.366      0.000       0.135       0.255
avg_rec                  0.0241      0.007      3.610      0.000       0.011       0.037
recXrecog_effort        -0.0236      0.010     -2.391      0.017      -0.043      -0.004
working_hours            0.0018      0.001      1.664      0.096      -0.000       0.004
firmsize                -0.0108      0.004     -2.566      0.010      -0.019      -0.003
tenure                  -0.0210      0.002     -9.660      0.000      -0.025      -0.017
tenure_squared           0.0330      0.005      7.065      0.000       0.024       0.042
age                      0.0421      0.006      6.749      0.000       0.030       0.054
age_squared             -0.1041      0.022     -4.718      0.000      -0.147      -0.061
years_educ               0.0194      0.007      2.775      0.006       0.006       0.033
commute_distance         0.0004      0.000      3.287      0.001       0.000       0.001
potential_experience     0.0007      0.000      2.773      0.006       0.000       0.001
mincer_residuals        -0.0620      0.021     -2.929      0.003      -0.103      -0.020
==============================================================================
Omnibus:                    15536.897   Durbin-Watson:                   1.807
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              322.124
Skew:                          -0.027   Prob(JB):                     1.13e-70
Kurtosis:                       1.699   Cond. No.                     1.16e+18
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 2.94e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Robustness Check 

Avg reciprocity measure over years



In [391]:
df_OLS_avg = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# specify columns which we need for regression by dropping everything else
df_OLS_avg.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction','avg_rec'], inplace = True)
# Load avg reciprocity measures over the years


# Max 'C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/rec_avgyears.csv'
# Maxie 
avg_reciprocity = pd.read_csv('C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/rec_avgyears.csv')
avg_reciprocity.reset_index(inplace=True)
avg_reciprocity.set_index(["pid","hid"], inplace=True)
avg_reciprocity.drop("index",axis=1 ,inplace =True)

df_avg_years = pd.merge(df_OLS_avg,avg_reciprocity, on=["pid","hid"])

# add interaction term
df_avg_years["recXrecog_effort"] = df_avg_years["recog_effort"] * df_avg_years["avg_rec"]
# drop missing data in regression dataframe
df_avg_years.dropna(inplace=True)

formula_main = 'turnover_intention ~ recog_effort + avg_rec + recXrecog_effort + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals'
# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=df_avg_years).fit(cov_type='cluster', cov_kwds={'groups': df_avg_years['sector']})

# Print the regression results
reg.summary()



c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-354-02c81fd1d7bc>:317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     753.9
Date:                Thu, 12 Jan 2023   Prob (F-statistic):           1.01e-58
Time:                        15:22:46   Log-Likelihood:                -3492.0
No. Observations:                5713   AIC:                             7010.
Df Residuals:                    5700   BIC:                             7096.
Df Model:                          12                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0349      0.007      4.838      0.000       0.021       0.049
recog_effort             0.1601      0.033      4.857      0.000       0.096       0.225
avg_rec                  0.0264      0.007      4.046      0.000       0.014       0.039
recXrecog_effort        -0.0156      0.011     -1.395      0.163      -0.038       0.006
working_hours            0.0021      0.001      1.859      0.063      -0.000       0.004
firmsize                -0.0115      0.004     -2.893      0.004      -0.019      -0.004
tenure                  -0.0221      0.002    -11.382      0.000      -0.026      -0.018
tenure_squared           0.0334      0.004      8.367      0.000       0.026       0.041
age                      0.0416      0.006      6.862      0.000       0.030       0.053
age_squared             -0.0980      0.021     -4.606      0.000      -0.140      -0.056
years_educ               0.0191      0.007      2.807      0.005       0.006       0.032
commute_distance         0.0004   9.98e-05      3.922      0.000       0.000       0.001
potential_experience     0.0006      0.000      2.484      0.013       0.000       0.001
mincer_residuals        -0.0775      0.019     -4.073      0.000      -0.115      -0.040
==============================================================================
Omnibus:                     7087.897   Durbin-Watson:                   1.803
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              372.516
Skew:                          -0.094   Prob(JB):                     1.29e-81
Kurtosis:                       1.763   Cond. No.                     1.00e+18
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is  5e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Also does not change much

##### Realized Turnover in 2007 -> turnover intentions = 1 else 0 with 2006 controls

In [ ]:
def realizedturnover(df_07):
    # gets people who realized turnover in 2006-2007
    jobchangers = df_07[df_07["new_job"] == '[1] Ja'].index

    observations_2006 = pd.read_csv('C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv')
    # some initial datamanagement
    observations_2006.reset_index(inplace=True)
    observations_2006.set_index(["pid","hid"], inplace=True)
    observations_2006.drop(columns=observations_2006.filter(regex='^syear').columns, inplace=True)
    observations_2006.drop("index",axis=1 ,inplace =True)
    # define people who changed their jobs
    jobstayers = observations_2006[~observations_2006.index.isin(jobchangers)]
    jobstayers["turnover_intention"] = 0
    # drop all to concatenate both: not elegant all which did not change job
    observations_2006.drop(observations_2006[~observations_2006.index.isin(jobchangers)].index, inplace=True)

    subset_2006 = pd.concat([jobstayers,observations_2006])

    return subset_2006

In [392]:
df_real = realizedturnover(df_07)
df_real.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'], inplace = True)
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month """
# add interaction term
df_real["recXrecog_effort"] = df_real["recog_effort"] * df_real["avg_rec"]
# drop missing data in regression dataframe
df_real.dropna(inplace=True)

formula_main = 'turnover_intention ~ recog_effort + avg_rec + recXrecog_effort + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals'
# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=df_avg_years).fit(cov_type='cluster', cov_kwds={'groups': df_avg_years['sector']})

# Print the regression results
reg.summary()

<ipython-input-330-5b5f3d0310db>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobstayers["turnover_intention"] = 0


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     753.9
Date:                Thu, 12 Jan 2023   Prob (F-statistic):           1.01e-58
Time:                        15:23:12   Log-Likelihood:                -3492.0
No. Observations:                5713   AIC:                             7010.
Df Residuals:                    5700   BIC:                             7096.
Df Model:                          12                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0349      0.007      4.838      0.000       0.021       0.049
recog_effort             0.1601      0.033      4.857      0.000       0.096       0.225
avg_rec                  0.0264      0.007      4.046      0.000       0.014       0.039
recXrecog_effort        -0.0156      0.011     -1.395      0.163      -0.038       0.006
working_hours            0.0021      0.001      1.859      0.063      -0.000       0.004
firmsize                -0.0115      0.004     -2.893      0.004      -0.019      -0.004
tenure                  -0.0221      0.002    -11.382      0.000      -0.026      -0.018
tenure_squared           0.0334      0.004      8.367      0.000       0.026       0.041
age                      0.0416      0.006      6.862      0.000       0.030       0.053
age_squared             -0.0980      0.021     -4.606      0.000      -0.140      -0.056
years_educ               0.0191      0.007      2.807      0.005       0.006       0.032
commute_distance         0.0004   9.98e-05      3.922      0.000       0.000       0.001
potential_experience     0.0006      0.000      2.484      0.013       0.000       0.001
mincer_residuals        -0.0775      0.019     -4.073      0.000      -0.115      -0.040
==============================================================================
Omnibus:                     7087.897   Durbin-Watson:                   1.803
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              372.516
Skew:                          -0.094   Prob(JB):                     1.29e-81
Kurtosis:                       1.763   Cond. No.                     1.00e+18
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is  5e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""